# QAMP-15: Building out Qiskit-QEC with the XP Stabilizer Formalism

Members: Dhruv Bhatnagar, __Ruihao Li__

Mentors: Grace Harper, Drew Vandeth (IBM)

Implementation based on the following work: <br>
M. A. Webster, B. J. Brown, and S. D. Bartlett, The XP Stabiliser Formalism: a Generalisation of the Pauli Stabiliser Formalism with Arbitrary Phases. *Quantum* __6__, 815 (2022).

## Setting things up

In [4]:
!git clone --branch qamp-qiskit-demodays https://github.com/qiskit-community/qiskit-qec.git /Users/ruihaoli/qiskit-qec

fatal: destination path '/Users/ruihaoli/qiskit-qec' already exists and is not an empty directory.


In [5]:
%cd ../qiskit-qec

/Users/ruihaoli/qiskit-qec


In [6]:
!pip install -r requirements.txt

## XP Operator Algebra

The XP operator algebra is a fundamental building block of the XP stabilizer formalism. Here we show how to express algebraic operations in terms of the vector representation and the generalized symplectic product. We also introduce the concepts of the degree and fundamental phase of XP operators. These concepts allow us to determine the eigenvalues and actions of the projectors of XP operators.

In the Pauli stabilizer formalism, one represents operators with the _symplectic representation_, i.e., with binary/boolean vectors $(\mathbf{x}, \mathbf{z})$ and a boolean phase $p$. Such representation is naturally generalized in the XP formalism.

Let $\mathbf{u} = (p | \mathbf{x} | \mathbf{z}) \in \mathbb Z\times \mathbb Z^n\times \mathbb Z^n$ be an integer vector of length $2n+1$. Recall that the XP operator of precision $N$ corresponding to $\mathbf{u}$ is defined as
$$
XP_N(\mathbf{u}) = \omega^p \bigotimes_{0\leq i < n} X^{\mathbf{x}_i} P^{\mathbf{z}_i},
$$
where
$$
\begin{split}
\omega &= \exp\left(\frac{\pi i}{N}\right), \\
P &= \text{diag}(1, \omega^2). \\
\end{split}
$$
Note that each component is periodic such that
$$
XP_N(p | \mathbf{x} | \mathbf{z}) = XP_N(p\ \text{mod}\ 2N | \mathbf{x}\ \text{mod}\ 2 | \mathbf{z}\ \text{mod}\ N).
$$
Therfore, we can define a __unique vector representation__ $(p | \mathbf{x} | \mathbf{z}) \in \mathbb Z_{2N}\times \mathbb Z^n_{2}\times \mathbb Z^n_N$ for each XP operator.

In [13]:
import numpy as np
from qiskit_qec.operators.xp_pauli import XPPauli
from qiskit_qec.operators.xp_pauli_list import XPPauliList

In [21]:
# XPPauli object
a = XPPauli(data=np.array([0, 3, 1, 6, 4, 3], dtype=np.int64), phase_exp=11, precision=4)
unique_a = a.unique_vector_rep()
print(unique_a.matrix)
print(unique_a.x)
print(unique_a.z)
print(unique_a._phase_exp)

[[0 1 1 2 0 3]]
[0 1 1]
[2 0 3]
[3]


In [18]:
# XPPauliList object
b = XPPauliList(
    data=np.array([[1, 0, 1, 1, 5, 3, 5, 4], [1, 0, 1, 0, 1, 5, 2, 0]], dtype=np.int64),
    phase_exp=np.array([4, 7]),
    precision=6,
)
unique_b = b.unique_vector_rep()
print(unique_b.matrix)
print(unique_b._phase_exp)

[[1 0 1 1 5 3 5 4]
 [1 0 1 0 1 5 2 0]]
[4 7]


When possible, we can also rescale the vector representation of an XP operator to one with a different precision $N'$. For example, for $XP_8(12|1110000|0040000)$, since the phase and $Z$ components are divisible by 4, we can rescale it to a precision-2 operator:
$$
XP_8(12|1110000|0040000) = XP_2(3|1110000|0010000).
$$

In [19]:
a = XPPauli(data=np.array([1, 1, 1, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0]), phase_exp=12, precision=8)
rescaled_a =  a.rescale_precision(new_precision=2, inplace=False)
print(rescaled_a.matrix)
print(rescaled_a._phase_exp)

[[1 1 1 0 0 0 0 0 0 1 0 0 0 0]]
[3]


In [20]:
# Case where it is not possible to rescale
a.rescale_precision(new_precision=3, inplace=False)

QiskitError: 'XP Operator cannot be expressed in new_precision.'

One important algebraic operation is __multiplication__ of two XP operators. To properly define the multiplication rule, we first introduce the __antisymmetric operator__ of precision $N$ corresponding to a vector $\mathbf{z}\in \mathbb Z^n$:
$$
D_{N}(\mathbf{z}) = XP_N(\sum_i \mathbf{z}_i | \mathbf{0} | -\mathbf{z}).
$$

In [23]:
a = XPPauli(data=np.array([1, 1, 2, 2, 1, 2, 3, 3]), phase_exp=0, precision=8)
antisym_op = a.antisymmetric_op(int_vec=a.z)
print(antisym_op.matrix)
print(antisym_op._phase_exp)

[[ 0  0  0  0 -1 -2 -3 -3]]
[9]


Then the product of two XP operators in vector form is given by
$$
XP_N(\mathbf{u}_1) XP_N(\mathbf{u}_2) = XP_N(\mathbf{u}_1 + \mathbf{u}_2) D_N(2\mathbf{u}_1 \mathbf{u}_2),
$$
where in this notation the arithmetic operations on vectors are component-wise in $\mathbb Z$, i.e.,
$$
\begin{split}
(\mathbf a + \mathbf b)[i] &= \mathbf a[i] + \mathbf b[i], \\
(\mathbf a \mathbf b)[i] &= \mathbf a[i] \mathbf b[i].
\end{split}
$$

In Qiskit-QEC, the multiplication operation is called `compose`.

In [25]:
a = XPPauli(data=np.array([0, 1, 0, 0, 2, 0]), phase_exp=6, precision=4)
b = XPPauli(data=np.array([1, 1, 1, 3, 3, 0]), phase_exp=2, precision=4)
product = a.compose(b, inplace=False)
print(product.matrix)
print(product._phase_exp)

[[1 0 1 3 3 0]]
[6]


In [26]:
# Multiplying two XPPauliList objects
a = XPPauliList(data=np.array([[1, 0, 1, 1, 5, 3, 5, 4], [1, 0, 1, 0, 1, 5, 2, 0]]), phase_exp=np.array([4, 7]), precision=6)
b = XPPauliList(data=np.array([[1, 0, 0, 1, 4, 1, 0, 1], [0, 1, 1, 0, 1, 3, 0, 5]]), phase_exp=np.array([11, 2]), precision=6)
product = a.compose(b, inplace=False)
print(product.matrix)
print(product._phase_exp)

[[0 0 1 0 1 4 5 3]
 [1 1 0 0 0 2 2 5]]
[ 1 11]
